In [1]:
#第十二章：pandas高级应用：
#12.1：分类数据
#从数组提取出不同的值，并分别计算频率：
import numpy as np;import pandas as pd
values = pd.Series(['apple','orange','apple','apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [2]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [3]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [4]:
#在数据仓库中，最好的方法时使用包含不同值得维表（Dimension Table），将主要的参数存储为引用维表整数键：
values = pd.Series([0,1,0,0]*2)
dim = pd.Series(['apple','orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [5]:
dim

0     apple
1    orange
dtype: object

In [6]:
#可用take方法存储原始得字符串Series：
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [9]:
#这种用整数表示的方法称为分类或字典编码表示法
#pandas得分类类型
#pandas有一个特殊的分类类型，用于保存使用整数分类表示法的数据
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit':fruits,'basket_id':np.arange(N),
                  'count': np.random.randint(3, 15,size=N),
                  'weight': np.random.uniform(0, 4,size=N)},
                 columns=['basket_id', 'fruit', 'count','weight'] 
                 )
df

,basket_id,fruit,count,weight
0,0,apple,13,1.086542
1,1,orange,10,1.257738
2,2,apple,14,2.114610
3,3,apple,8,1.217470
4,4,apple,8,2.303200
5,5,orange,6,1.740421
6,6,apple,4,3.037012
7,7,apple,13,1.437350


In [10]:
#df['fruit']是一个python字符串对象得数组，可调用它，将其转换为分类：
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [11]:
#fruit-cat得值不是Numpy数组，而是一个pandas.Categorical实例：
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [13]:
#分类对象有categories和codes属性：
c.categories

Index(['apple', 'orange'], dtype='object')

In [14]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [15]:
#可将DataFrame得列通过分配转换结果，转换为分类：
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [16]:
#可直接从python序列中直接创建pandas.Categorical:
my_categories = pd.Categorical(['foo','bar','baz','foo','bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [17]:
#从其它源获得了分类编码，还可以使用from_codes构造器：
categories = ['foo','bar','baz']
codes = [0,1,2,0,0,1]
my_cats_2 = pd.Categorical.from_codes(codes,categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [18]:
#当使用from_codes或其它的构造器时，可以指定分类一个有意义的顺序：
ordered_cat = pd.Categorical.from_codes(codes,categories,ordered = True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [20]:
#无序的分类实例可通过as_ordered排序：
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [21]:
#用分类进行计算：
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [22]:
#计算这个数据的分位面元，提取一些统计信息：
bins = pd.qcut(draws,4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [23]:
#虽然有用，与确切的样本分位数与分位的名称相比，不利于生成汇总，可使用labels参数qcut：
bins = pd.qcut(draws,4,labels = ['Q1','Q2','Q3','Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [24]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [25]:
#加上标签的面元分类不包含数据面元边界的信息，因此可以使用groupby提取一些汇总信息：
bins = pd.Series(bins,name = 'quartile')
results = (pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [26]:
#分位数列保存了原始的面元分类信息，包括排序：
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [27]:
#用分类提高性能：
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo','bar','baz','qux']*(N//4))
#将标签转换为分类：
categories = labels.astype('category')
#可看见标签使用的内存远比分类多：
labels.memory_usage()

80000080

In [28]:
categories.memory_usage()

10000272

In [29]:
%time  _ = labels.astype('category')

Wall time: 401 ms


In [31]:
#分类方法：
#包含分类数据的Series有一些特殊的方法，类似于Series.str字符串方法。它还提供了方便的分类和编码的使用方法
s = pd.Series(['a','b','c','d']*2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [32]:
#特别的cat属性提供了分类方法的入口：
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [33]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [35]:
#如果实际数据分类集中超出了数据中的四个值，可使用set_categories方法改变他们：
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [36]:
#数据虽没变，但新的分类将反映在其操作中，value_counts表示分类：
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [37]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [38]:
#过滤完大DataFrame或Series后，许多分类可能不会出现在数据中，可使用remove_unused_categories方法删除没看到的分类：
cat_s3 = cat_s[cat_s.isin(['a','b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [40]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [41]:
#为建模创建虚拟变量
#当你使用统计或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。
#包括创建一个不同类别的列的DataFrame；这些列包含给定分类的1s，其它为0。
cat_s = pd.Series(['a','b','c','d']*2,dtype = 'category')

In [42]:
#pandas.get_dummies函数可以转换这个分类数据为包含虚拟变量的DataFrame：
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


In [43]:
#12.2.Groupby高级应用
#分组转换和“解封”Groupby
#转换的方法不仅有apply方法，还有transfporm方法，其与apply很像，但是对使用的函数有一定限制：
#可产生向分组形状广播标量值
#可产生一个和输入组形状相同的对象
#不能修改输入
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [44]:
#按键进行分组
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [45]:
#假设想产生一个和df['value']形状相同的Series，但值替换为按键分组的平均
#值。我们可以传递函数lambda x: x.mean()进行转换：
g.transform(lambda x:x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [46]:
#对于内置的聚合函数，可以传递一个字符串假名作为GroupBy的agg方法：
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [48]:
#与apply类似，transform的函数会返回Series，但是结果必须与输入大小相同：用lambda函数将每个分组乘以2：
g.transform(lambda x:x*2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [49]:
#计算每个分组的降序排名：
g.transform(lambda x:x.rank(ascending = False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [50]:
#由简单聚合构造的分组转换函数：
def normalize(x):
    return(x - x.mean())/x.std()
#用transform或apply可以获得等价的结果
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [51]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [52]:
#内置的聚合函数：mean或sum，快速很多，这允许进行一个所谓的解封（unwrapped）分组操作：
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [53]:
normalized = (df['value'] - g.transform('mean'))/g.transform('std')
normalized
#解封分组操作可能包括多个分组聚合，但是矢量化操作还是会带来收益。

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [55]:
#分组的时间重采样
N = 15
times = pd.date_range('2017-05-20 00:00',freq = '1min',periods = N)
df = pd.DataFrame({'time':times,'value':np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [56]:
#可以用time作为索引，然后重采样：
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [62]:
#假设DataFrame包含多个时间序列，用一个额外的分组键的列进行标记：
df2 = pd.DataFrame({'time': times.repeat(3),'key': np.tile(['a', 'b', 'c'], N),'value': np.arange(N * 3.)})
df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [63]:
#对每个key值进行相同的重采样，可引入pandas.TimeGrouper对象：
time_key = pd.TimeGrouper('5min')
#然后设定时间索引，用key和time_key分组，然后聚合：
resampled = (df2.set_index('time').groupby(['key',time_key]).sum())
resampled

E:\Anaconda\Setup\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  


value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [65]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


In [ ]:
#使用TimeGrouper的限制是时间必须是Series或DataFrame的索引。

In [66]:
#12.3.链式编程技术
df = load_data()
df2 = df[df['col2']< 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
#虽没导入数据，DataFrame.assign方法时一个df[k] = v形式的列分配方法，其不是就地修改对象，而是返回
#新的修改过的DataFrame，与下式等价：
#Usual non-functional way
df2 = df.copy()
df2['k'] = v
#Functional assign way
df2 = df.assian(k = v)
#就地分配可能会比assign快，但是assign可以方便地进行链式编程：
result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean()).groupby('key').col1_demeaned.std())
#管道方法

NameError: name 'load_data' is not defined